In [1]:
import pandas as pd

In [2]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
from bokeh.io import show
from bokeh.plotting import figure

In [4]:
final_data = pd.read_csv('final_output.csv')
# final_data.loc[final_data['TEAM'] == 'DAL'].PLAYER.unique()

In [5]:
from bokeh.models import ColumnDataSource
source = ColumnDataSource(dict(
    x=final_data.PC1,
    y=final_data.PC2,
    player=final_data.PLAYER,
    team=final_data.TEAM,
    age=final_data.AGE,
    games_played=final_data.GP,
    clusters=final_data.db_label,
    plus_minus=final_data['+/-'],
    points=final_data.PTS
    )
)

In [6]:
from bokeh.models import HoverTool

In [7]:
from bokeh.models import LinearInterpolator, LinearColorMapper, ColorBar, BoxZoomTool, ResetTool
from bokeh.palettes import Plasma256

color_mapper = LinearColorMapper(
    palette = Plasma256,
    low=final_data.PTS.min(), 
    high=final_data.PTS.max()
)

size_mapper = LinearInterpolator(
    x=[final_data.PTS.min(), final_data.PTS.max()],
    y=[2, 40]
)

In [8]:
from ipywidgets import interact
from bokeh.io import push_notebook

def update(player):
    cluster = int(final_data.loc[final_data['PLAYER'] == player].db_label)
    new_data = dict(
        x=final_data.loc[final_data['db_label'] == cluster].PC1,
        y=final_data.loc[final_data['db_label'] == cluster].PC2,
        player=final_data.loc[final_data['db_label'] == cluster].PLAYER,
        team=final_data.loc[final_data['db_label'] == cluster].TEAM,
        age=final_data.loc[final_data['db_label'] == cluster].AGE,
        games_played=final_data.loc[final_data['db_label'] == cluster].GP,
        clusters=final_data.loc[final_data['db_label'] == cluster].db_label,
        plus_minus=final_data.loc[final_data['db_label'] == cluster]['+/-'],
        points=final_data.loc[final_data['db_label'] == cluster].PTS
    )
    source.data = new_data
    p.title.text = str('Basketballers similar to %s during the 2015/16 season' % player)
    push_notebook()

In [9]:
hover = HoverTool(tooltips='@player', show_arrow=False)
p = figure(tools=[hover,BoxZoomTool(), ResetTool()],
          title='Basketballers from the 2015/16 season',
          plot_width=900)
p.circle(x='x', y='y',
         size={'field': 'points', 'transform': size_mapper},
         color={'field': 'points', 'transform': color_mapper},
         alpha=0.5,
         source=source
)
p.add_layout(ColorBar(color_mapper=color_mapper, location=(0, 0), orientation='horizontal'), 'below')
show(p, notebook_handle=True)

In [10]:
interact(update, player=list(final_data.PLAYER.unique()))